In [40]:
import numpy as np
import pandas as pd
import cv2
import os

image3 = cv2.imread("data/memorial.hdr",cv2.IMREAD_ANYDEPTH)


In [41]:
gray_image3 = np.mean(image3,axis=2)
############*Log image***********8#
log_image = np.log(gray_image3)
np.min(log_image)
log_image_t = (255/(np.max(log_image)-np.min(log_image)))*log_image -(255/(np.max(log_image)-np.min(log_image)))*np.min(log_image)


In [42]:
######Filters 
def gaussian_kernel_1d(size:int,sigma:float)->np.array:
    distance = np.square(np.linspace(-(size//2),size//2,size))
    kernel = np.exp(-0.5 * distance / np.square(sigma) )
    return kernel/np.sum(kernel)
np.convolve([0,1,4,5],[1,2,3],mode='same')

def convolve_1D(A:np.array,kernel:np.array)->np.array:
    C=[]
    for row in A:
        c_rows = np.convolve(row,kernel,mode='same')
        C.append(c_rows)
    return np.array(C)
def full_convolution(A:np.array,kernel:np.array)->np.array:
    A1 = convolve_1D(A,kernel).T
    result = convolve_1D(A1,kernel).T
    return result